In [2]:
%config IPCompleter.greedy=True

In [62]:
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.techindicators import TechIndicators
from matplotlib.pyplot import figure
import matplotlib.pyplot as plt
import time
import pandas as pd

In [4]:
# Your key here
key = 'GKLUA07WU7MDOTHC'

In [117]:
# List of companies
list_companies = ['AAPL','MSFT','AMZN','FB','BRK.B','JNJ','GOOG','JPM','V','PG']

# Chose your output format, or default to JSON (python dict)
ts = TimeSeries(key, output_format='pandas')
ti = TechIndicators(key)


In [118]:
# Sector for each company
dict_sector = {'AAPL':'Technology','MSFT':'Technology', 'AMZN':'Retail',
               'FB':'Technology','BRK.B':'Financial Services','JNJ':'Pharmaceuticals',
               'GOOG':'Technology','JPM':'Financial Services','V':'Financial Services',
                'PG':'Industrial'}

sector_data = []
for k,v in dict_sector.items():
    sector_data.append(v)
    
s_sector = pd.Series(data = sector_data, index = list_companies)
s_sector
    

AAPL             Technology
MSFT             Technology
AMZN                 Retail
FB               Technology
BRK.B    Financial Services
JNJ         Pharmaceuticals
GOOG             Technology
JPM      Financial Services
V        Financial Services
PG               Industrial
dtype: object

In [127]:
df_list = []
for company in list_companies:
    data, metadata = ts.get_monthly(symbol=company)
    data = pd.DataFrame(data['4. close'])
    data.rename(columns = {'4. close':company}, inplace = True)
    df_list.append(data)
#     print(df_list)
    time.sleep(21)
    

In [140]:
df_prices = pd.concat(df_list,axis=1,sort=True)
df_prices
# for i in df_prices.index:
#     print(i)

,AAPL,MSFT,AMZN,FB,BRK.B,JNJ,GOOG,JPM,V,PG
date,,,,,,,,,,
2000-05-31,84.00,62.56,49.94,NaN,1886.00,89.50,NaN,74.69,NaN,66.50
2000-06-30,52.38,80.00,36.31,NaN,1760.00,101.90,NaN,46.06,NaN,56.75
2000-07-31,50.81,69.81,30.13,NaN,1825.00,93.06,NaN,49.81,NaN,57.00
2000-08-31,60.94,69.81,41.50,NaN,1913.00,91.96,NaN,55.88,NaN,61.83
2000-09-29,25.75,60.31,38.44,NaN,2070.00,93.94,NaN,46.19,NaN,67.00
...,...,...,...,...,...,...,...,...,...,...
2019-12-31,293.65,157.70,1847.84,205.25,226.50,145.87,1337.02,139.40,187.90,124.90
2020-01-31,309.51,170.23,2008.72,201.91,224.43,148.87,1434.23,132.36,198.97,124.62
2020-02-28,273.36,162.01,1883.75,192.47,206.34,134.48,1339.33,116.11,181.76,113.23


In [129]:
# close prices for all tickers on given date
df_prices.loc['2000-08-31']

# tickers with highest 2 close prices
list(df_prices.loc['2000-08-31'].nlargest(2).index)


set(s_sector.loc[list(df_prices.loc['2000-08-31'].nlargest(2).index)])
# series_sector.loc[df_prices.loc['2020-04-09'].nlargest(2).index]

{'Financial Services', 'Pharmaceuticals'}

In [122]:
def date_top_industries(prices, sector, date, top_n):
    """
    Get the set of the top industries for the date
    
    Parameters
    ----------
    prices : DataFrame
        Prices for each ticker and date
    sector : Series
        Sector name for each ticker
    date : Date
        Date to get the top performers
    top_n : int
        Number of top performers to get
    
    Returns
    -------
    top_industries : set
        Top industries for the date
    """
    # TODO: Implement Function
#     return set(sector.loc[prices.loc[date].nlargest(top_n).index])
    close_prices = prices.loc[date]
    top_n_tickers = list(close_prices.nlargest(top_n).index)
    sector_lookup = sector.loc[top_n_tickers]
    
    return set(sector_lookup)

In [139]:
date_top_industries(df_prices, s_sector, '2008-09-30', 2)

{'Financial Services', 'Technology'}